In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [3]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content="I'm sorry, I am not able to provide real-time weather information. I recommend checking a reliable weather website or app for up-to-date weather information for Munich.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 17, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C0V8nw4qsWuI8KgGY86u06AVnAQEr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6ca766d7-aa84-43f4-8645-b2ea8b3a984e-0', usage_metadata={'input_tokens': 17, 'output_tokens': 33, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
from langchain_core.tools import tool


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "munich" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "munich" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."


tools = [get_weather, check_seating_availability]

In [5]:
llm_with_tools = llm.bind_tools(tools)

In [6]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mG5qGlssAcevim7OBTYcecHn', 'function': {'arguments': '{"location":"munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 84, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C0V8vSad3eLfWVPzvYAWE7NIgkE8D', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2ec066fe-d9a3-4980-a446-f6c7c222c5f5-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'munich'}, 'id': 'call_mG5qGlssAcevim7OBTYcecHn', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 15, 'total_tokens': 99, 'input_token_details': {'audio': 0, 

In [7]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': 'call_mG5qGlssAcevim7OBTYcecHn',
  'type': 'tool_call'}]

In [8]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tkPgTIMpnA375S6MFJ5DAf7P', 'function': {'arguments': '{"location": "munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_SYT8YsA91xVA10Z9RG7pbJlc', 'function': {'arguments': '{"location": "munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 92, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C0V8yLmsesaUHogUj7CpSVmUFfwKO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--93d5ca4c-f049-44e4-99a9-0e6d495718d7-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'muni

In [9]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': 'call_tkPgTIMpnA375S6MFJ5DAf7P',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'munich', 'seating_type': 'outdoor'},
  'id': 'call_SYT8YsA91xVA10Z9RG7pbJlc',
  'type': 'tool_call'}]

In [10]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? Do you still have seats outdoor available?"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [11]:
messages

[HumanMessage(content='How will the weather be in munich today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n36znu5J5fnI3yvldWK6S6pF', 'function': {'arguments': '{"location": "munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_EgIN4vdI8NDiAaGnLPbBEHHI', 'function': {'arguments': '{"location": "munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 92, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C0V9PhATj63j1KQ0H75b06yb4PBXI', 'service_tier': 'default', 'finish

In [12]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability,
}

In [13]:
llm_output.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': 'call_n36znu5J5fnI3yvldWK6S6pF',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'munich', 'seating_type': 'outdoor'},
  'id': 'call_EgIN4vdI8NDiAaGnLPbBEHHI',
  'type': 'tool_call'}]

In [14]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [15]:
messages

[HumanMessage(content='How will the weather be in munich today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n36znu5J5fnI3yvldWK6S6pF', 'function': {'arguments': '{"location": "munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_EgIN4vdI8NDiAaGnLPbBEHHI', 'function': {'arguments': '{"location": "munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 92, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C0V9PhATj63j1KQ0H75b06yb4PBXI', 'service_tier': 'default', 'finish

In [16]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today is 15 degrees Celsius and cloudy. And yes, we still have seats available outdoors.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 177, 'total_tokens': 201, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C0V9zyVXEcLDGqXFXS2tJIRFjnHmM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--05c74a1d-b72a-444c-afc8-270970143f95-0', usage_metadata={'input_tokens': 177, 'output_tokens': 24, 'total_tokens': 201, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})